<a href="https://colab.research.google.com/github/chainiqedu/chainiqedu/blob/main/volmex_csv_data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Volmex Labs - LPs/Traders Processing

Processing ERC20 Transactions across Ethereum, Polygon & Arbitrum

In [1]:
# Import Libraries
import numpy as np
import pandas as pd

## Processing/Helper Functions

In [2]:
def label_tx_type(df, address):
  df['Type'] = np.where(df['To'] == address, 'IN', 'OUT')
  return df

In [3]:
def concat_tx_label_type(df):
  df_action_labelled = df[['Txhash', 'Type', 'UnixTimestamp']].groupby(['Txhash', 'UnixTimestamp'], as_index=False).agg({'Type': '-'.join})
  return df_action_labelled

In [4]:
def tx_type_df(df):

    if (df['Type'] == 'IN' or df['Type'] == 'IN-IN' or df['Type'] == 'IN-IN-IN-IN'):
        return 'Add Liquidity'
    elif (df['Type'] == 'OUT-OUT' or df['Type'] == 'IN-OUT-OUT-OUT' or df['Type'] == 'IN-OUT-OUT-OUT-IN-OUT-OUT-OUT'):
        return 'Remove Liquidity'
    elif (df['Type'] == 'OUT-IN' or df['Type'] == 'IN-OUT' or df['Type'] == 'IN-OUT-IN-OUT'):
        return 'Swap'
    elif (df['Type'] == 'OUT-IN-IN-IN' or df['Type'] == 'IN-OUT-IN-IN' or df['Type'] == 'IN-OUT-IN-IN-IN-OUT-IN-IN' or df['Type'] == 'IN-OUT-IN-OUT-IN-IN'):
        return 'Liquidity Added and Swap'
    elif (df['Type'] == 'IN-OUT-OUT-OUT-IN-OUT'):
        return 'Liquidity Removed and Swap'

In [5]:
def tx_action_type_named (df):
  df['tx_type_label'] = df.apply(tx_type_df, axis=1)
  return df[['Txhash', 'tx_type_label']]

In [6]:
def tx_type_labels_lp_trader_csv(tx_type_labels_df, lps_traders_df):
  return lps_traders_df.merge(tx_type_labels_df, on='Txhash', how='left')

In [7]:
def run_processing(df, address):
  df_labels = label_tx_type(df, address)
  tx_type_labelled = concat_tx_label_type(df_labels)
  tx_type_labels = tx_action_type_named(tx_type_labelled)
  lps_traders_labels_df = tx_type_labels_lp_trader_csv(tx_type_labels, df_labels)
  return lps_traders_labels_df

## Ethereum Processing

In [184]:
# Read CSV's for Uniswap Volmex Pools
ethv_usdc_lps_traders_df = pd.read_csv('/content/export-address-token-0x113ecd438bff3e63a95b0b9d18c38bbf066db5a0.csv', index_col=False)
iethv_usdc_lps_traders_df = pd.read_csv('/content/export-address-token-0x46631c701eae6c6f1880cdd6898675bbf20886bb.csv', index_col=False)
btcv_usdc_lps_traders_df = pd.read_csv('/content/export-address-token-0x97693242c5703900fc7651c9164a334ee5ddfa2d.csv', index_col=False)
ibtcv_usdc_lps_traders_df = pd.read_csv('/content/export-address-token-0x08008a915911271e2b09c08e1cae1ad1c48b933f.csv', index_col=False)

In [185]:
# Perform Processing
ethv_usdc_lps_traders_processed_df = run_processing(ethv_usdc_lps_traders_df, '0x113ecd438bff3e63a95b0b9d18c38bbf066db5a0')
iethv_usdc_lps_traders_processed_df = run_processing(iethv_usdc_lps_traders_df, '0x46631c701eae6c6f1880cdd6898675bbf20886bb')
btcv_usdc_lps_traders_processed_df = run_processing(btcv_usdc_lps_traders_df, '0x97693242c5703900fc7651c9164a334ee5ddfa2d')
ibtcv_usdc_lps_traders_processed_df = run_processing(ibtcv_usdc_lps_traders_df, '0x08008a915911271e2b09c08e1cae1ad1c48b933f')

In [186]:
# Write to CSV
ethv_usdc_lps_traders_processed_df.to_csv('ethv_usdc_uniswap_v3.csv')
iethv_usdc_lps_traders_processed_df.to_csv('iethv_usdc_uniswap_v3.csv')
btcv_usdc_lps_traders_processed_df.to_csv('btcv_usdc_uniswap_v3.csv')
ibtcv_usdc_lps_traders_processed_df.to_csv('ibtcv_usdc_uniswap_v3.csv')

## Polygon Processing

In [187]:
# Read CSV's for Quickswap Volmex Pools
part_one_ethv_usdc_lps_traders_df = pd.read_csv('/content/part_1_export-address-token-0xf0dcde1a4e404e050276517a5b1b82f3c4a04f01.csv', index_col=False)
part_two_ethv_usdc_lps_traders_df = pd.read_csv('/content/part_2_export-address-token-0xf0dcde1a4e404e050276517a5b1b82f3c4a04f01 (1).csv', index_col=False)
ethv_usdc_lps_traders_df = pd.concat([part_one_ethv_usdc_lps_traders_df, part_two_ethv_usdc_lps_traders_df]).drop_duplicates()

part_one_iethv_usdc_lps_traders_df = pd.read_csv('/content/part_1_export-address-token-0x384a154b8def8b7a8867458a38a894eaf9969ce2.csv', index_col=False)
part_two_iethv_usdc_lps_traders_df = pd.read_csv('/content/part_2_export-address-token-0x384a154b8def8b7a8867458a38a894eaf9969ce2 (1).csv', index_col=False)
iethv_usdc_lps_traders_df = pd.concat([part_one_iethv_usdc_lps_traders_df, part_two_iethv_usdc_lps_traders_df]).drop_duplicates()

btcv_usdc_lps_traders_df = pd.read_csv('/content/export-address-token-0x7d8667001de41872c5ccde94a0957b0061702bad.csv', index_col=False)
ibtcv_usdc_lps_traders_df = pd.read_csv('/content/export-address-token-0xf8a85f679fb256859a1310c59a05de0848c66f18.csv', index_col=False)

In [188]:
# Perform Processing
ethv_usdc_lps_traders_processed_df = run_processing(ethv_usdc_lps_traders_df, '0xf0dcde1a4e404e050276517a5b1b82f3c4a04f01')
iethv_usdc_lps_traders_processed_df = run_processing(iethv_usdc_lps_traders_df, '0x384a154b8def8b7a8867458a38a894eaf9969ce2')
btcv_usdc_lps_traders_processed_df = run_processing(btcv_usdc_lps_traders_df, '0x7d8667001de41872c5ccde94a0957b0061702bad')
ibtcv_usdc_lps_traders_processed_df = run_processing(ibtcv_usdc_lps_traders_df, '0xf8a85f679fb256859a1310c59a05de0848c66f18')

In [189]:
# Write to CSV
ethv_usdc_lps_traders_processed_df.to_csv('ethv_usdc_quickswap_v3.csv')
iethv_usdc_lps_traders_processed_df.to_csv('iethv_usdc_quickswap_v3.csv')
btcv_usdc_lps_traders_processed_df.to_csv('btcv_usdc_quickswap_v3.csv')
ibtcv_usdc_lps_traders_processed_df.to_csv('ibtcv_usdc_quickswap_v3.csv')

## Arbitrum Processing

In [19]:
# Read CSV's for Arbitrum Volmex Pools
ethv_usdc_lps_traders_df = pd.read_csv('/content/export-address-token-0x8278591bbbbe7d86e783507e7f6320ffc9c021e5.csv', index_col=False)
iethv_usdc_lps_traders_df = pd.read_csv('/content/export-address-token-0x5497ca4be41d2ae0222cc4c41c43d189ed894ffd.csv', index_col=False)
btcv_usdc_lps_traders_df = pd.read_csv('/content/export-address-token-0x3396e2cb9e4f1eb4fc06028ab07a8db0ef63b486.csv', index_col=False)
ibtcv_usdc_lps_traders_df = pd.read_csv('/content/export-address-token-0xc80956377cea73f64aaa3613d6692eabdf4fd4c8.csv', index_col=False)

In [20]:
# Perform processing
ethv_usdc_lps_traders_processed_df = run_processing(ethv_usdc_lps_traders_df, '0x8278591bbbbe7d86e783507e7f6320ffc9c021e5')
iethv_usdc_lps_traders_processed_df = run_processing(iethv_usdc_lps_traders_df, '0x5497ca4be41d2ae0222cc4c41c43d189ed894ffd')
btcv_usdc_lps_traders_processed_df = run_processing(btcv_usdc_lps_traders_df, '0x3396e2cb9e4f1eb4fc06028ab07a8db0ef63b486')
ibtcv_usdc_lps_traders_processed_df = run_processing(ibtcv_usdc_lps_traders_df, '0xc80956377cea73f64aaa3613d6692eabdf4fd4c8')

In [21]:
# Write to CSV
ethv_usdc_lps_traders_processed_df.to_csv('ethv_usdc_arbitrum_uniswap_v3.csv')
iethv_usdc_lps_traders_processed_df.to_csv('iethv_usdc_arbitrum_uniswap_v3.csv')
btcv_usdc_lps_traders_processed_df.to_csv('btcv_usdc_arbitrum_uniswap_v3.csv')
ibtcv_usdc_lps_traders_processed_df.to_csv('ibtcv_usdc_arbitrum_uniswap_v3.csv')